In [ ]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, START, END

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)
my_api_key = os.getenv("OPENAI_API_KEY")
# print (f'Key is {my_api_key}')


client = OpenAI(api_key=my_api_key)


# --- Shared State ---
class LibraryState(TypedDict):
    question: Optional[str]
    faq_answer: Optional[str]
    checkout_info: Optional[str]
    final_answer: Optional[str]


def ClassifierAgent(state: LibraryState):
    
    print(f"ClassifierAgent Initial state: {state}")

    # Build the LLM messages
    message_to_llm = [
        {"role": "system", "content": '''You are a classifier agent in a library system. 
        Decide if the user is asking about book availability/checkout or about library FAQs. 
        Reply with JSON containing keys: faq_answer and checkout_info.'''},
        {"role": "user", "content": f"Question: {state['question']}"}
    ]
    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=message_to_llm,
        # temperature=0.2,   # keep it deterministic for classification
        # max_tokens=150,
    )
    print (response)
    # Extract the content from the response
    answer = response.choices[0].message.content
    # Ideally, parse as JSON — here assuming model returns a dict-like string
    try:
        import json
        parsed = json.loads(answer)
        print(f"Raw response: {answer}")
        print(f"Parsed response: {parsed}")

        return {
            "faq_answer": parsed.get("faq_answer", ""),
            "checkout_info": parsed.get("checkout_info", "")
        }
    except Exception:
        # fallback if LLM gives plain text
        return {"faq_answer": answer, "checkout_info": ""}

def FAQAgent(state: LibraryState):
    
    print(f"FAQAgent Initial state: {state}")
    if not state.get("faq_answer"):
        return {"faq_answer": "Default FAQ: Library rules apply"}
    return {"faq_answer": state["faq_answer"]}


def CheckoutAgent(state: LibraryState):
    print(f"CheckoutAgent Initial state: {state}")
    if not state.get("checkout_info"):
        return {"checkout_info": "Checkout info: Not requested"}
    return { "checkout_info": state["checkout_info"]}


def ResponseAgent(state: LibraryState):
    print(f"CheckouResponseAgent Initial state: {state}")
  
    message_to_llm = [
        {"role": "system", "content": '''You are a response builder for the library application.
          Please combine the FAQ answer and checkout info into a coherent response to the user's question.
        '''},
        {"role": "user", "content": f"FAQ: {state['faq_answer']}\nCheckout Info: {state['checkout_info']}\nQuestion: {state['question']}"}
    ]

    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=message_to_llm,
        # temperature=0.2,   # keep it deterministic for classification
        # max_tokens=150,
    )

    print (response)
    # Extract the content from the response
    final_answer = response.choices[0].message.content


    return {"final_answer": final_answer}


# --- Build the Graph ---
builder = StateGraph(LibraryState)
builder.add_node("ClassifierAgent", ClassifierAgent)
builder.add_node("FAQAgent", FAQAgent)
builder.add_node("CheckoutAgent", CheckoutAgent)
builder.add_node("ResponseAgent", ResponseAgent)

builder.add_edge(START, "ClassifierAgent")
builder.add_edge("ClassifierAgent", "FAQAgent")
builder.add_edge("ClassifierAgent", "CheckoutAgent")
builder.add_edge("FAQAgent", "ResponseAgent")
builder.add_edge("CheckoutAgent", "ResponseAgent")
builder.add_edge("ResponseAgent", END)

graph = builder.compile()

result = graph.invoke({"question": "When does library open?"})
print("\n--- Final Answer ---")
print(result["final_answer"])

ClassifierAgent Initial state: {'question': 'When does library open?'}
ChatCompletion(id='chatcmpl-CSQ7FAOLGOeEbVnqIiMu5rxxHVY4G', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "faq_answer": "Library hours vary by branch. Please tell me which branch you’re asking about, or check the Hours page on our website for the most up-to-date opening times. If you’d like, I can look up the hours for a specific branch.",\n  "checkout_info": null\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760889609, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=777, prompt_tokens=59, total_tokens=836, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=704, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails

In [4]:


# # --- Visualize ---
# print(graph.get_graph().draw_ascii())
# graph.get_graph().draw_png("images/agentic_ai_library.png")
# print("Graph saved as agentic_ai_library.png")

In [11]:
# --- Run ---
result = graph.invoke({"question": "When does library open?"})
print("\n--- Final Answer ---")
print(result["final_answer"])


ClassifierAgent ran
state: {'question': 'When does library open?'}
ChatCompletion(id='chatcmpl-CDDw5hU1p1SMaV3rapBjn6VpnU4p5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"faq_answer": "The library opens at 9:00 AM and closes at 5:00 PM from Monday to Saturday. It is closed on Sundays.", "checkout_info": null}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757267869, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_7c233bf9d1', usage=CompletionUsage(completion_tokens=39, prompt_tokens=60, total_tokens=99, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
Raw response: {"faq_answer": "The library opens at 9:00 AM and closes at 5:00 PM from 

In [12]:
result = graph.invoke({"question": "Is The Hobbit available?"})
print("\n--- Final Answer ---")
print(result["final_answer"])


ClassifierAgent ran
state: {'question': 'Is The Hobbit available?'}
ChatCompletion(id='chatcmpl-CDDwDyrvTkY8mrGyjeHj5bph25qOZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "faq_answer": null,\n  "checkout_info": "The Hobbit is available for checkout. Please visit the library or check online to reserve a copy."\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757267877, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_04d3664870', usage=CompletionUsage(completion_tokens=34, prompt_tokens=60, total_tokens=94, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
Raw response: {
  "faq_answer": null,
  "checkout_info": "The Hobbit is availab